![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [24]:
!pip install transformers
!pip install evaluate

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
# Attempt to load the dataset again with automatic delimiter detection
import pandas as pd
car_reviews_df = pd.read_csv('data/car_reviews.csv', delimiter=';')
car_reviews_df.head()


,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


## Sentiment Analysis with Pre-Trained LLM

In [26]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score

# Load reviews
car_reviews_df = pd.read_csv('data/car_reviews.csv', delimiter=';')
reviews = car_reviews_df['Review'].tolist()

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Perform sentiment analysis
predicted_results = sentiment_pipeline(reviews)

# Map predicted labels to {0,1}
predicted_labels = [1 if result['label'] == 'POSITIVE' else 0 for result in predicted_results]

# Assuming you have actual sentiment labels in binary form {0,1}
actual_labels = car_reviews_df['Class'].map({'POSITIVE': 1, 'NEGATIVE': 0}).tolist()

# Calculate accuracy and F1 score
accuracy_result = accuracy_score(actual_labels, predicted_labels)
f1_result = f1_score(actual_labels, predicted_labels)

accuracy_result, f1_result

(0.8, 0.8571428571428571)

## Translation and BLEU Score Calculation: 
This involves translating text and then calculating the BLEU score to assess translation quality.

In [27]:
import re
from nltk.translate.bleu_score import sentence_bleu
from transformers import pipeline

car_reviews_df = pd.read_csv('data/car_reviews.csv', delimiter=';')

# Extract the first review
first_review = car_reviews_df.loc[0, 'Review']
# Assuming the first review has been extracted as first_review
first_two_sentences = ' '.join(re.split(r'(?<=[.!?]) +', first_review)[:2])
translation_pipeline = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")
# Translate the first two sentences
translated_review = translation_pipeline(first_two_sentences)[0]['translation_text']

# Load reference translations
with open('data/reference_translations.txt', 'r', encoding='utf-8') as file:
    reference_translations = [line.strip().split() for line in file.readlines()]

# Calculate BLEU score
bleu_score = sentence_bleu(reference_translations, translated_review.split())
bleu_score, translated_review

(0.7532821983227991,
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.')

## Extractive Question Answering: 
This task entails using a specific language model to answer questions based on the context provided by a text passage. 

In [28]:
# Define context and question for the 2nd review
context = car_reviews_df.loc[1, 'Review']
question = "What did he like about the brand?"

qa_pipeline = pipeline("question-answering", model="deepset/minilm-uncased-squad2")
# Get the answer
answer = qa_pipeline(question=question, context=context)
answer = answer['answer'] if 'answer' in answer else "Answer not found"
answer

'ride quality, reliability'

In [29]:
# Import auto classes (optional: can be solved via pipelines too)
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
import torch 

# Instantiate model and tokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = reviews[1]
print(f"Context:\n{context}")
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

# Perform inference and extract answer from raw outputs
with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]
# Decode and show answer
answer = tokenizer.decode(answer_span)
print("Answer: ", answer)

Context:
The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Answer:  ride quality, reliability


## Summarization: 
Summarizing text into a specified token length involves using summarization models.

In [30]:
text_to_summarize = car_reviews_df.iloc[-1]['Review']
print(f"Original text:\n{text_to_summarize}")

# Load summarization pipeline and perform inference
model_name = "cnicu/t5-small-booksum"
summarizer = pipeline("summarization", model=model_name)
outputs = summarizer(text_to_summarize, max_length=53, min_length=25)
summarized_text = outputs[0]['summary_text']
print(f"Summarized text:\n{summarized_text}")

Original text:
I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of la